In [ ]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import imutils
import time
import dlib
import cv2



In [ ]:
datFile =  "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(datFile)

In [ ]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
	# return the eye aspect ratio
	return ear


In [ ]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
def detect_drowsy(img_path): #detects on a single image
  prediction = 0
  EYE_AR_THRESH = 0.20
  EYE_AR_CONSEC_FRAMES = 0
  # initialize the frame counter as well as a boolean used to
  # indicate if the alarm is going off
  COUNTER = 0
  ALARM_ON = False
  img = cv2.imread(img_path)
  img = imutils.resize(img, width=450)
  #img = cv2.flip(img,1)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # detect faces in the grayscale frame
  rects = detector(gray, 0)
    # loop over the face detections
  for rect in rects:
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    # extract the left and right eye coordinates, then use the
    # coordinates to compute the eye aspect ratio for both eyes
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    # average the eye aspect ratio together for both eyes
    ear = (leftEAR + rightEAR) / 2.0
    # compute the convex hull for the left and right eye, then
    # visualize each of the eyes
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContours(img, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(img, [rightEyeHull], -1, (0, 255, 0), 1)
    if ear < EYE_AR_THRESH:
      COUNTER += 1
      # if the eyes were closed for a sufficient number of
      # then sound the alarm
      if COUNTER >= EYE_AR_CONSEC_FRAMES:
        # draw an alarm on the frame
        prediction = 1
        return prediction
        cv2.putText(img, "DROWSINESS ALERT!", (10, 30),
          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    # otherwise, the eye aspect ratio is not below the blink
    # threshold, so reset the counter and alarm
    else:
      COUNTER = 0
      ALARM_ON = False
    # draw the computed eye aspect ratio on the frame to help
    # with debugging and setting the correct eye aspect ratio
    # thresholds and frame counters
    cv2.putText(img, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    return prediction

In [ ]:
def detect_drowsy_frames(list_img): #runs on a list of images
  result =[]
  consecutive = []
  for i in range(len(list_img)):
    predict = detect_drowsy(list_img[i])
    result.append(predict)
  for j in range(1,len(result)):
    if result[0]==result[1] and result[0]==1:
      consecutive.append(result[0])
    if result[j] == result[j-1] and result[j]==1:
      consecutive.append(result[j])
  if sum(consecutive) >= 5:
    return 'drowsy'
  else:
    return 'awake'